## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_path_list = glob.glob('/kaggle/input/135bert-base-last/' + '*.h5')
model_path_list += glob.glob('/kaggle/input/135bert-base-best/' + '*.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_1.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_2.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_3.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_4.h5
/kaggle/input/135bert-base-best/135-BERT_base_uncased_model_fold_5.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_1_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_2_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_3_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_4_last_epoch.h5
/kaggle/input/135bert-base-last/135-BERT_base_uncased_model_fold_5_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.915407,0.585986,0.253737,0.473990,0.607013,0.531151,0.653277,0.631053,0.654172,...,0.889116,0.877294,0.534111,0.942182,0.935478,0.749339,0.087217,0.060595,0.774386,0.893437
1,46,0.885027,0.538459,0.003729,0.820688,0.808804,0.941099,0.586101,0.497367,0.043642,...,0.710167,0.952437,0.668357,0.972761,0.982822,0.876966,0.886154,0.082042,0.095093,0.892966
2,70,0.919559,0.652219,0.020460,0.781273,0.888530,0.938205,0.599291,0.526753,0.197719,...,0.844712,0.942992,0.620222,0.972815,0.976303,0.861268,0.133235,0.077366,0.845080,0.910910
3,132,0.889049,0.428273,0.004355,0.727160,0.775450,0.907991,0.548211,0.430804,0.102253,...,0.733830,0.950763,0.668530,0.975308,0.985100,0.900625,0.831777,0.157064,0.430946,0.911334
4,200,0.913335,0.444638,0.038953,0.821334,0.738264,0.834119,0.643133,0.604879,0.092254,...,0.713245,0.909767,0.628907,0.955596,0.953123,0.820929,0.181190,0.111068,0.592516,0.895674


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.890566,0.592345,0.035566,0.722491,0.810630,0.848460,0.585465,0.500067,0.223997,...,0.810493,0.931383,0.657797,0.963630,0.970967,0.861776,0.499867,0.129541,0.509641,0.907016
std,2812.670060,0.044469,0.120405,0.067487,0.105032,0.098609,0.124977,0.049392,0.085746,0.178915,...,0.075681,0.024230,0.046470,0.012331,0.012741,0.039636,0.309749,0.062252,0.264332,0.019387
min,39.000000,0.761737,0.369814,0.001565,0.257396,0.364866,0.273754,0.474269,0.358358,0.004118,...,0.592200,0.844139,0.534111,0.917323,0.901922,0.738277,0.005763,0.004640,0.038066,0.829201
25%,2572.000000,0.860721,0.484967,0.006519,0.663133,0.769892,0.827252,0.548161,0.436700,0.083520,...,0.751334,0.916992,0.623758,0.955796,0.964045,0.837178,0.172857,0.087441,0.284868,0.894421
50%,5093.000000,0.891215,0.585852,0.010267,0.728183,0.820138,0.894853,0.577154,0.475322,0.162027,...,0.813561,0.935009,0.655919,0.965009,0.973967,0.866845,0.558879,0.128964,0.487976,0.908506
75%,7482.000000,0.926440,0.681395,0.027303,0.789198,0.876920,0.925238,0.614824,0.547655,0.327618,...,0.876021,0.949459,0.688953,0.972607,0.979634,0.891677,0.789639,0.168110,0.737129,0.920313
max,9640.000000,0.978843,0.915459,0.568003,0.965282,0.979810,0.974290,0.732786,0.791550,0.811361,...,0.963315,0.982816,0.822834,0.988924,0.993092,0.966102,0.959313,0.311225,0.987761,0.957803
